In [ ]:
# Install required packages
!pip install datasets pandas numpy torch tqdm faiss-cpu sentence-transformers transformers openpyxl kaggle

# Import necessary libraries
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
import faiss
from sentence_transformers import SentenceTransformer
import json
import os
